이 코딩은 지정한 날짜에서부터 n일 까지 네이버 뉴스의 모든 뉴스를 가져오는 코딩이다.

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import time
import datetime
from datetime import timedelta
import pickle

today=datetime.date(2021,11,18)
n=10


def request_url(url):
    sleep_time=5
    num=5
    for _ in range(num):
        try:
            html=urlopen(url)
            code = html.status  
            time.sleep(sleep_time)
            if code==200: 
                bs=BeautifulSoup(html,'lxml')
                return bs
                break
            
        except Exception as e:
            print("url_page",e)
            time.sleep(30)
    return -1
    
def list_page(date,page):
    url="https://news.naver.com/main/list.naver?mode=LSD&mid=sec&sid1=001&date={}&page={}".format(date,page)
    bs=request_url(url.format(date,page))
    return bs

def news_page(url):
    news={}
    news['url']=url
#     print('url',url)
    body_pro=True
    for _ in range(3):
        bs=request_url(url)
        if bs!=-1:
            try:
                head=bs.select_one("div.article_info")

                news['title']=head.select_one("h3#articleTitle").text
                news['day']=head.select_one("span.t11").text
                body=bs.select_one("div#articleBodyContents").text
                body=body.replace("\n","").replace("\xa0","").replace("\t",'')
                news['body']=body.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}","")
                sections=bs.select("em.guide_categorization_item")
                news['sections']=[section.text for section in sections]
            except Exception as e:
                print("뉴스 바디", e)
            else:
                body_pro=False
                break
    if not body_pro:    
        try:   #부가적
            company=bs.select_one("div.article_header a[href]")
            news['company_url']=company['href']
            news["company"]=company.select_one("img[title]")['title']
        except Exception as e:
            print("부가적 문제1",e)
            print(url)
            fail_log.append([3,url])
            
        try:   #부가적
            news['reporter']=bs.select_one("p.b_text").text.strip()
        except Exception as e:
            print("부가적 문제2",e)
            print(url)
            fail_log.append([4,url])
    else:
        fail_log.append([2,url])

    return news




for j in range(1,n):  
    try:
        fail_log=[]
    #     today=datetime.datetime.today()
        yesterday=today - timedelta(days=j)
        y_day=yesterday.strftime('%Y%m%d')
        print('day : ',y_day)

        keep=False
        for _ in range(3):
            bs=list_page(y_day,10000)
            if bs!=-1:
                try:
                    max_page_num=bs.select_one('div.paging strong').text
                    print(max_page_num)
                except Exception as e:
                    print("max_num",e)
                    time.sleep(20)
                else:
                    keep=True
                    break
            else:
                time.sleep(7)

        if keep:
            news_pd=pd.DataFrame([])
            for i in range(1,int(max_page_num)):
                if i%100==0:
                    print(i,"/",max_page_num)
                lis=True
                for _ in range(2):
                    bs=list_page(y_day,i)
                    if bs!=-1:
                        news_list=bs.select('div.list_body.newsflash_body li')
                        if len(news_list)!=0:
                            for new in news_list:

                                url=new.select_one('a[href]')
                                if url!=None:
                                    dic=news_page(url['href'])
                                    if dic!={}:
                                        news_pd=news_pd.append(dic,ignore_index=True)
                            news_pd.to_csv("news_pd_{}.csv".format(y_day),index=False)
                            lis=False
                            break
                if lis:
                    fail_log.append([1,y_day,i])
        else:
            fail_log.append([0,y_day])
    except KeyboardInterrupt as e:
        print("key")
    finally:
        with open('fail_log_{}.pkl'.format(y_day),'wb') as f:
            pickle.dump(fail_log,f)

day :  20211028
1302
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=077&aid=0005367727
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=077&aid=0005367726
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=077&aid=0005367725
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=077&aid=0005367723
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=077&aid=0005367722
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=077&aid=0005367720
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=077&aid=0005367719
부가적 문제2 'None

부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=028&aid=0002565735
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=055&aid=0000930087
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=055&aid=0000930085
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=077&aid=0005367630
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=077&aid=0005367629
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=077&aid=0005367628
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=028&aid=0002565734
부가적 문제2 'NoneType' object has no a

부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=421&aid=0005688414
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=029&aid=0002703430
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=029&aid=0002703429
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=421&aid=0005688404
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=421&aid=0005688403
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=421&aid=0005688401
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=421&aid=0005688400
부가적 문제2 'NoneType' object has no a

부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=057&aid=0001617493
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=057&aid=0001617492
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=057&aid=0001617491
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=003&aid=0010799098
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=028&aid=0002565718
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=028&aid=0002565717
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=028&aid=0002565716
부가적 문제2 'NoneType' object has no a

부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=082&aid=0001125151
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=082&aid=0001125149
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=421&aid=0005688263
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=082&aid=0001125148
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=014&aid=0004730765
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=014&aid=0004730761
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=055&aid=0000930045
부가적 문제2 'NoneType' object has no a

...

부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=009&aid=0004865129
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=009&aid=0004865128
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=057&aid=0001614252
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=003&aid=0010771579
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=421&aid=0005659170
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=421&aid=0005659169
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=421&aid=0005659161
부가적 문제2 'NoneType' object has no a

부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=009&aid=0004865093
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=009&aid=0004865092
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=009&aid=0004865091
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=009&aid=0004865087
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=003&aid=0010771348
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=003&aid=0010771344
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=421&aid=0005658975
부가적 문제2 'NoneType' object has no a

부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=421&aid=0005658867
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=421&aid=0005658866
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=421&aid=0005658865
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=421&aid=0005658864
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=214&aid=0001153915
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=214&aid=0001153914
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=214&aid=0001153913
부가적 문제2 'NoneType' object has no a

부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=081&aid=0003222540
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=081&aid=0003222533
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=081&aid=0003222521
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=081&aid=0003222512
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=028&aid=0002563971
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=028&aid=0002563969
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=028&aid=0002563968
부가적 문제2 'NoneType' object has no a

부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=023&aid=0003646418
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=023&aid=0003646415
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=023&aid=0003646412
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=023&aid=0003646409
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=020&aid=0003387376
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=020&aid=0003387374
부가적 문제2 'NoneType' object has no attribute 'text'
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=001&oid=023&aid=0003646406
부가적 문제2 'NoneType' object has no a